# Graph Coloring Problem, Local Search Metaheuristics

The **graph coloring problem** can be described mathematically as follows:

Let **G = (V, E)** be an undirected graph, where:

- **V** is the set of vertices (or nodes) of the graph.
- **E** is the set of edges, where each edge *e = (u, v)* represents a connection between two vertices *u* and *v*.

---

### Objective

The goal is to assign a color to each vertex in **V** such that no two adjacent vertices share the same color. The **chromatic number** χ(G) of the graph G is the minimum number of colors required to color the vertices of the graph in this way. Formally:

**χ(G) = min { k : there exists a function f : V → {1, 2, ..., k} such that for all (u, v) ∈ E, f(u) ≠ f(v) }**

Where:

- *f : V → {1, 2, ..., k}* is a coloring function that assigns a color (represented by a number) to each vertex.
- The constraint *f(u) ≠ f(v)* ensures that adjacent vertices do not share the same color.

---

### Computational Complexity

Determining the chromatic number χ(G) of a graph is a well-known **NP-hard** problem. This means that, in general, there is no efficient algorithm that can find the exact chromatic number for arbitrary graphs in polynomial time, unless P = NP.

Thus, finding the optimal solution (i.e., the exact chromatic number) is computationally expensive, especially for large graphs.

---

### Local Search Metaheuristics

Given the computational hardness of the graph coloring problem, **local search methods** offer powerful alternatives. These algorithms do not guarantee optimality but can find near-optimal solutions efficiently, especially on large graphs.

---

### Simulated Annealing (SA)

**Simulated Annealing** is a probabilistic metaheuristic inspired by the annealing process in metallurgy. It explores the solution space by accepting both improving and, with a certain probability, worsening moves to escape local minima.

- **Initialization**: Start with an initial coloring using a fixed number of colors *k*.
- **Neighborhood**: The neighborhood typically consists of solutions obtained by changing the color of one vertex.
- **Acceptance Criterion**:
  - If the move reduces the number of conflicts, it is accepted.
  - If it increases conflicts, it may still be accepted with a probability **P = exp(-ΔE / T)**, where **ΔE** is the change in the objective function and **T** is the current temperature.
- **Cooling Schedule**: Temperature **T** decreases over time (e.g., geometric cooling: **T_new = α × T**, where **α** is a constant in (0, 1)).
- **Termination**: The algorithm stops after a fixed number of iterations or when the temperature becomes sufficiently low.

> Simulated Annealing is particularly effective for escaping local minima and provides a good balance between exploration and exploitation.

---

### Tabu Search (TS)

**Tabu Search** is a local search technique that uses memory structures to avoid cycling and entrapment in local optima.

- **Initialization**: Begin with an initial solution using *k* colors.
- **Neighborhood**: Defined by single-vertex recolorings that modify the current solution slightly.
- **Tabu List**: A short-term memory that records recent moves (e.g., changing vertex *v* from color *c1* to *c2*) and forbids them for a number of iterations.
- **Aspiration Criterion**: A tabu move can be allowed if it leads to a solution better than the best found so far.
- **Long-Term Memory**: Optional strategies include intensification (searching promising regions more thoroughly) and diversification (exploring new areas).

> Tabu Search is particularly useful in guiding the search away from cycles and previously explored poor regions of the search space.

---

### Variable Neighborhood Search (VNS)

**Variable Neighborhood Search** systematically changes the neighborhood structure during the search to overcome local optima.

- **Basic Idea**: Local optima in one neighborhood may not be local optima in another.
- **Procedure**:
  1. Start with an initial solution and a set of neighborhoods **N1, N2, ..., Nk**.
  2. For each neighborhood **Ni**:
     - Generate a random solution **s' ∈ Ni(s)**.
     - Apply local search to find a local optimum **s''**.
     - If **s''** is better than the current solution **s**, accept it and reset **i** to 1.
     - Otherwise, move to the next neighborhood **i + 1**.
- **Neighborhood Structures**: Typically include different types of vertex recoloring moves (e.g., single-vertex, Kempe chain-based, or color-swap moves).
- **Termination**: Stops when a time limit is reached or no improvement is found across all neighborhoods.

> VNS is effective for escaping local optima by strategically changing the scope of local search.


## Imports and Dependencies

The following libraries are used:
- `math`: For mathematical operations (e.g., exponential in Simulated Annealing).
- `time`: To enforce time limits in the solver.
- `random`: For random selections in heuristics and metaheuristics.

In [1]:
!pip install optuna
!pip install optuna[visualization]

In [2]:
import random
import math
import time
import optuna.visualization as vis
from collections import defaultdict

In [3]:
import logging
import warnings
logging.getLogger('optuna').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=FutureWarning)

## Définition de la classe `Graph`

Nous définissons une classe `Graph` pour représenter un graphe non orienté à l'aide d'une liste d'adjacence. Cette structure permet une recherche efficace des voisins et une manipulation simple des sommets et arêtes.

Fonctionnalités principales :
- `add_vertex(v)` : ajoute un sommet `v` au graphe.
- `add_edge(u, v)` : ajoute une arête non orientée entre `u` et `v`.
- `get_neighbors(v)` : retourne les voisins du sommet `v`.
- `get_degree(v)` : retourne le degré du sommet `v`.
- `from_dimacs(file_path)` : méthode de classe permettant de charger un graphe à partir d’un fichier au format **DIMACS**.

In [4]:
class Graph:
    """
    Classe représentant un graphe avec une structure de stockage optimisée.
    Utilise des listes d'adjacence pour une recherche efficace des voisins.
    """
    def __init__(self):
        self.num_vertices = 0
        self.num_edges = 0
        self.adjacency_list = {}  # Liste d'adjacence: {sommet: [voisins]}
        self.degrees = {}  # Degré de chaque sommet: {sommet: degré}
    
    def add_vertex(self, v):
        """Ajoute un sommet au graphe s'il n'existe pas déjà."""
        if v not in self.adjacency_list:
            self.adjacency_list[v] = set()
            self.degrees[v] = 0
            self.num_vertices += 1
    
    def add_edge(self, u, v):
        """Ajoute une arête entre les sommets u et v."""
        # Ajouter les sommets s'ils n'existent pas déjà
        self.add_vertex(u)
        self.add_vertex(v)
        
        # Ajouter l'arête si elle n'existe pas déjà
        if v not in self.adjacency_list[u]:
            self.adjacency_list[u].add(v)
            self.adjacency_list[v].add(u)
            self.degrees[u] += 1
            self.degrees[v] += 1
            self.num_edges += 1
    
    def get_neighbors(self, v):
        """Retourne la liste des voisins du sommet v."""
        return self.adjacency_list.get(v, set())
    
    def get_degree(self, v):
        """Retourne le degré du sommet v."""
        return self.degrees.get(v, 0)
    
    @classmethod
    def from_dimacs(cls, file_path):
        """
        Construit un graphe à partir d'un fichier au format DIMACS.
        
        Format DIMACS:
        - Lignes commençant par 'c' sont des commentaires
        - Ligne 'p edge n m' déclare un graphe avec n sommets et m arêtes
        - Lignes 'e u v' représentent une arête entre les sommets u et v
        """
        graph = cls()
        
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                
                # Ignorer les lignes vides et les commentaires
                if not line or line.startswith('c'):
                    continue
                
                parts = line.split()
                
                # Déclaration du problème
                if parts[0] == 'p' and parts[1] == 'edge':
                    n = int(parts[2])  # Nombre de sommets
                    # Pré-initialiser les sommets de 1 à n
                    for i in range(1, n+1):
                        graph.add_vertex(i)
                
                # Déclaration d'une arête
                elif parts[0] == 'e':
                    u, v = int(parts[1]), int(parts[2])
                    graph.add_edge(u, v)
        
        return graph

# Graph Coloring Algorithms Implementation : Approach 1

## Core Components
- **Conflict Detection**: Helper functions to identify and count coloring conflicts
- **Coloring Initialization**: 
  - Greedy coloring
  - Welsh-Powell heuristic
  - DSATUR (Degree of Saturation) heuristic

## Local Search Methods
1. **Simulated Annealing**
   - Temperature-based probabilistic acceptance
   - Smart neighborhood selection focusing on conflicting vertices
   - Dynamic cooling schedule

2. **Tabu Search**
   - Tabu list with variable tenure
   - Aspiration criteria
   - Diversification strategy

3. **Variable Neighborhood Search (VNS)**
   - Multiple shaking neighborhoods
   - Local search intensification
   - Periodic color reduction attempts

### Helper Functions

These functions support the graph coloring algorithms by:
- Counting conflicts (edges with same-colored endpoints).
- Identifying vertices involved in conflicts.
- Validating a coloring (ensuring no conflicts).

In [5]:
# Helper functions for graph coloring
def count_conflicts(graph, coloring):
    """
    Counts the number of conflicts in a given coloring.
    A conflict is an edge whose endpoints have the same color.
    """
    conflicts = 0
    edges_checked = set()
    
    for u in graph.adjacency_list:
        for v in graph.get_neighbors(u):
            if u < v:  # Check each edge only once
                edge = (u, v)
                if edge not in edges_checked:
                    edges_checked.add(edge)
                    if coloring.get(u) == coloring.get(v):
                        conflicts += 1
    
    return conflicts

def get_conflicting_vertices(graph, coloring, conflict_cache=None):
    """
    Identifies vertices in conflict in a given coloring.
    Returns a set of vertices that have at least one conflict.
    """
    if conflict_cache is not None and id(coloring) in conflict_cache:
        return conflict_cache[id(coloring)]
        
    conflicting_vertices = set()
    
    for u in graph.adjacency_list:
        u_color = coloring.get(u)
        for v in graph.get_neighbors(u):
            if u_color == coloring.get(v):
                conflicting_vertices.add(u)
                break
                
    if conflict_cache is not None:
        conflict_cache[id(coloring)] = conflicting_vertices
    return conflicting_vertices

def is_valid_coloring(graph, coloring, conflict_cache=None):
    """Checks if a coloring is valid (no adjacent vertices with the same color)."""
    return len(get_conflicting_vertices(graph, coloring, conflict_cache)) == 0

### Initialization Heuristics

These heuristics generate initial colorings:
- **Greedy Coloring**: Assigns the smallest possible color to each vertex based on neighbors' colors.
- **Welsh-Powell Coloring**: Prioritizes vertices by degree, assigning colors greedily.
- **DSatur Coloring**: Selects vertices based on saturation degree (number of unique neighbor colors).

In [6]:
# Initialization heuristics
def greedy_coloring(graph):
    """
    Optimized simple greedy coloring.
    Pre-allocates an array of available colors to improve performance.
    """
    coloring = {}
    vertices = list(graph.adjacency_list.keys())
    max_degree = max(len(graph.get_neighbors(v)) for v in vertices)
    available_colors = [True] * (max_degree + 2)  # +2 to ensure an available color
    
    for v in vertices:
        # Mark colors used by neighbors
        for neighbor in graph.get_neighbors(v):
            if neighbor in coloring:
                available_colors[coloring[neighbor]] = False
        
        # Find the first available color
        color = 1
        while color <= max_degree + 1 and not available_colors[color]:
            color += 1
        
        coloring[v] = color
        
        # Reset the array for the next vertex
        for neighbor in graph.get_neighbors(v):
            if neighbor in coloring:
                available_colors[coloring[neighbor]] = True
    
    return coloring

def welsh_powell_coloring(graph):
    """
    Optimized Welsh-Powell coloring.
    """
    coloring = {}
    
    # Sort vertices in decreasing order of degree - use a list of tuples for sorting
    vertices_by_degree = [(v, graph.get_degree(v)) for v in graph.adjacency_list.keys()]
    vertices_by_degree.sort(key=lambda x: x[1], reverse=True)
    
    max_degree = vertices_by_degree[0][1]
    available_colors = [True] * (max_degree + 2)
    
    for v, _ in vertices_by_degree:
        # Mark colors used by neighbors
        for neighbor in graph.get_neighbors(v):
            if neighbor in coloring:
                available_colors[coloring[neighbor]] = False
        
        # Find the first available color
        color = 1
        while color <= max_degree + 1 and not available_colors[color]:
            color += 1
        
        coloring[v] = color
        
        # Reset the array for the next vertex
        for neighbor in graph.get_neighbors(v):
            if neighbor in coloring:
                available_colors[coloring[neighbor]] = True
    
    return coloring

def dsatur_coloring(graph):
    """
    Optimized DSatur (Degree of Saturation) coloring.
    Uses a more efficient data structure to track saturation.
    """
    coloring = {}
    uncolored = set(graph.adjacency_list.keys())
    
    # Initialize data structures for saturation tracking
    saturation = {v: 0 for v in uncolored}
    neighbors_colors = {v: set() for v in uncolored}  # Colors of neighbors
    
    # Select initial vertex with maximum degree
    first_vertex = max(uncolored, key=lambda v: graph.get_degree(v))
    
    while uncolored:
        # Select the vertex with the highest saturation
        if len(uncolored) == len(graph.adjacency_list):
            # First vertex - use the one with maximum degree
            v = first_vertex
        else:
            v = max(uncolored, key=lambda u: (saturation[u], graph.get_degree(u)))
        
        # Determine the minimum available color
        used_colors = neighbors_colors[v]
        color = 1
        while color in used_colors:
            color += 1
        
        # Assign the color and update structures
        coloring[v] = color
        uncolored.remove(v)
        
        # Update saturation of uncolored neighbors
        for neighbor in graph.get_neighbors(v):
            if neighbor in uncolored:
                neighbors_colors[neighbor].add(color)
                saturation[neighbor] = len(neighbors_colors[neighbor])
    
    return coloring


### Local Search Methods

These methods refine colorings by exploring neighboring solutions:
- **Shake**: Generates a new coloring by modifying the current one (e.g., changing a vertex's color or swapping colors).
- **Local Search**: Iteratively improves a coloring by resolving conflicts for individual vertices.

In [7]:
# Local search methods
def shake(graph, coloring, k, max_color, neighbors=None, conflict_cache=None):
    """
    Generates a solution in the neighborhood k of the current solution (optimized).
    """
    new_coloring = coloring.copy()
    
    # Different neighborhood structures according to k
    if k == 1:
        # Color change for a vertex - prefer conflicting vertices
        conflicting = get_conflicting_vertices(graph, coloring, conflict_cache)
        if conflicting:
            v = random.choice(list(conflicting))
        else:
            v = random.choice(list(graph.adjacency_list.keys()))
            
        current_color = new_coloring[v]
        
        # Choose a color not used by neighbors if possible
        if neighbors:
            used_colors = {new_coloring[n] for n in neighbors[v]}
            available_colors = [c for c in range(1, max_color + 1) if c != current_color and c not in used_colors]
            if available_colors:
                new_coloring[v] = random.choice(available_colors)
                return new_coloring
        
        # Fallback: random color
        new_color = random.randint(1, max_color)
        while new_color == current_color:
            new_color = random.randint(1, max_color)
        new_coloring[v] = new_color
    
    elif k == 2:
        # Swap colors between two vertices - target conflicting vertices
        conflicting = get_conflicting_vertices(graph, coloring, conflict_cache)
        vertices = list(graph.adjacency_list.keys())
        
        if conflicting and len(conflicting) >= 2:
            v1, v2 = random.sample(list(conflicting), 2)
        elif conflicting:
            v1 = random.choice(list(conflicting))
            v2 = random.choice([v for v in vertices if v != v1])
        else:
            v1, v2 = random.sample(vertices, 2)
            
        new_coloring[v1], new_coloring[v2] = new_coloring[v2], new_coloring[v1]
    
    elif k == 3:
        # Recoloring of a subset, targeting conflicting vertices
        conflicting = get_conflicting_vertices(graph, coloring, conflict_cache)
        vertices = list(conflicting) if conflicting else list(graph.adjacency_list.keys())
        
        subset_size = min(5, len(vertices))
        if subset_size > 0:
            subset = random.sample(vertices, subset_size)
            
            for v in subset:
                if neighbors:
                    # Try to avoid neighbors' colors
                    used_colors = {new_coloring[n] for n in neighbors[v]}
                    available_colors = [c for c in range(1, max_color + 1) if c not in used_colors]
                    if available_colors:
                        new_coloring[v] = random.choice(available_colors)
                        continue
                
                # Fallback: random color
                new_coloring[v] = random.randint(1, max_color)
    
    # Invalidate cache
    if conflict_cache is not None:
        conflict_cache.pop(id(new_coloring), None)
    return new_coloring

def local_search(graph, coloring, max_color, neighbors=None, conflict_cache=None, max_iterations=100):
    """
    Performs an optimized local search from a given coloring.
    """
    if neighbors is None:
        neighbors = {v: graph.get_neighbors(v) for v in graph.adjacency_list}
        
    current_coloring = coloring.copy()
    current_conflicts = count_conflicts(graph, current_coloring)
    
    for _ in range(max_iterations):
        if current_conflicts == 0:
            break
            
        # Identify conflicting vertices
        conflicting_vertices = get_conflicting_vertices(graph, current_coloring, conflict_cache)
        
        if not conflicting_vertices:
            break
        
        improved = False
        
        # Try to resolve conflicts efficiently
        for v in conflicting_vertices:
            current_color = current_coloring[v]
            
            # Calculate colors used by neighbors
            used_colors = set()
            for neighbor in neighbors[v]:
                used_colors.add(current_coloring[neighbor])
            
            # Try each possible color
            for new_color in range(1, max_color + 1):
                if new_color != current_color and new_color not in used_colors:
                    # Apply the new color
                    current_coloring[v] = new_color
                    
                    # Check if the number of conflicts has decreased
                    new_conflicts = 0
                    for u in graph.adjacency_list:
                        for w in neighbors[u]:
                            if u < w and current_coloring[u] == current_coloring[w]:
                                new_conflicts += 1
                                if new_conflicts >= current_conflicts:
                                    break
                        if new_conflicts >= current_conflicts:
                            break
                    
                    if new_conflicts < current_conflicts:
                        current_conflicts = new_conflicts
                        improved = True
                        # Update cache
                        if conflict_cache is not None:
                            conflict_cache.pop(id(current_coloring), None)
                        break
                    else:
                        # Undo the change
                        current_coloring[v] = current_color
        
        if not improved:
            break
    
    return current_coloring, current_conflicts

### Implementation Of Metaheuristic Algorithms

These algorithms improve initial colorings using advanced search strategies:
- **Simulated Annealing**: Uses probabilistic acceptance of worse solutions to escape local optima.
- **Tabu Search**: Avoids revisiting recent solutions using a tabu list.
- **Variable Neighborhood Search (VNS)**: Explores multiple neighborhood structures to improve solutions.

In [8]:
def simulated_annealing(graph, initial_coloring=None, initial_k=None, max_iterations=10000, 
                        initial_temp=1.0, cooling_rate=0.99, min_temp=0.001):
    """
    Optimized simulated annealing method for graph coloring.
    """
    # Initialization
    if initial_coloring is None:
        if initial_k is None:
            initial_coloring = dsatur_coloring(graph)
            initial_k = max(initial_coloring.values())
        else:
            initial_coloring = {v: random.randint(1, initial_k) for v in graph.adjacency_list}
    
    current_coloring = initial_coloring.copy()
    current_conflicts = count_conflicts(graph, current_coloring)
    
    best_coloring = current_coloring.copy()
    best_conflicts = current_conflicts
    
    # Pre-compute adjacency lists for faster access
    neighbors = {v: graph.get_neighbors(v) for v in graph.adjacency_list}
    
    conflict_cache = {}
    temp = initial_temp
    iteration = 0
    
    while iteration < max_iterations and temp > min_temp and best_conflicts > 0:
        # Select a vertex to modify - prefer conflicting vertices
        conflicting = get_conflicting_vertices(graph, current_coloring, conflict_cache)
        if conflicting and random.random() < 0.9:  # 90% chance to choose a conflicting vertex
            v = random.choice(list(conflicting))
        else:
            v = random.choice(list(graph.adjacency_list.keys()))
        
        current_color = current_coloring[v]
        
        # Choose a new color - smarter strategy
        if random.random() < 0.5:  # 50% chance to try a color not used by neighbors
            used_colors = {current_coloring[n] for n in neighbors[v]}
            available_colors = [c for c in range(1, initial_k + 1) if c != current_color and c not in used_colors]
            if available_colors:
                new_color = random.choice(available_colors)
            else:
                new_color = random.randint(1, initial_k)
                while new_color == current_color:
                    new_color = random.randint(1, initial_k)
        else:
            new_color = random.randint(1, initial_k)
            while new_color == current_color:
                new_color = random.randint(1, initial_k)
        
        # Calculate energy delta efficiently
        delta_e = 0
        for neighbor in neighbors[v]:
            if current_coloring[neighbor] == current_color:
                delta_e -= 1  # Removing a conflict
            if current_coloring[neighbor] == new_color:
                delta_e += 1  # Adding a conflict
        
        # Accept or reject the new solution
        if delta_e <= 0 or random.random() < math.exp(-delta_e / temp):
            current_coloring[v] = new_color
            current_conflicts += delta_e
            
            # Update conflict cache
            conflict_cache.pop(id(current_coloring), None)
            
            # Update best solution if necessary
            if current_conflicts < best_conflicts:
                best_coloring = current_coloring.copy()
                best_conflicts = current_conflicts
        
        # Cool down
        temp *= cooling_rate
        iteration += 1
    
    return best_coloring, best_conflicts

### Hyper Parameter Optimisation

In [9]:
import optuna
from functools import partial

def objective(trial, graph, initial_coloring=None, initial_k=None):
    # Sample hyperparameters
    initial_temp = trial.suggest_float("initial_temp", 0.1, 5.0, log=True)
    cooling_rate = trial.suggest_float("cooling_rate", 0.90, 0.999)
    min_temp = trial.suggest_float("min_temp", 1e-5, 0.1, log=True)
    max_iterations = trial.suggest_int("max_iterations", 1000, 20000, step=1000)

    coloring, conflicts = simulated_annealing(
        graph,
        initial_coloring=initial_coloring,
        initial_k=initial_k,
        max_iterations=max_iterations,
        initial_temp=initial_temp,
        cooling_rate=cooling_rate,
        min_temp=min_temp
    )

    # Goal: minimize the number of conflicts
    return conflicts

def run_optuna_optimization(graph, n_trials=50, initial_coloring=None, initial_k=None):
    study = optuna.create_study(direction="minimize")
    study.optimize(partial(objective, graph=graph, initial_coloring=initial_coloring, initial_k=initial_k), n_trials=n_trials)

    print("Best trial:")
    print("  Value (conflicts):", study.best_value)
    print("  Params:", study.best_params)
    return study


# Load the graph
try:
    graph_file = '/kaggle/input/optimisation/dsjc250.5.col'
    graph = Graph.from_dimacs(graph_file)
    print(f"Graph loaded: {graph.num_vertices} vertices, {graph.num_edges} edges")
except Exception as e:
    print(f"Error loading graph: {e}")
study = run_optuna_optimization(graph, n_trials=100,initial_k=28)
best_params = study.best_params

coloring, conflicts = simulated_annealing(
    graph,
    initial_k=max(dsatur_coloring(graph).values()),
    **best_params
)

Graph loaded: 250 vertices, 15668 edges
Best trial:
  Value (conflicts): 53.0
  Params: {'initial_temp': 2.2651850148207564, 'cooling_rate': 0.9988739483784393, 'min_temp': 1.12986037289236e-05, 'max_iterations': 13000}


In [10]:
# Parallel coordinate plot
vis.plot_parallel_coordinate(study).show()

In [11]:
# Optimization history
vis.plot_optimization_history(study).show()

In [12]:
# Parameter importance
vis.plot_param_importances(study).show()

In [13]:
def tabu_search(graph, initial_coloring=None, initial_k=None, max_iterations=1000, tabu_tenure=7):
    """
    Optimized tabu search method for graph coloring.
    """
    # Initialization
    if initial_coloring is None:
        if initial_k is None:
            initial_coloring = dsatur_coloring(graph)
            initial_k = max(initial_coloring.values())
        else:
            initial_coloring = {v: random.randint(1, initial_k) for v in graph.adjacency_list}
    
    current_coloring = initial_coloring.copy()
    current_conflicts = count_conflicts(graph, current_coloring)
    
    best_coloring = current_coloring.copy()
    best_conflicts = current_conflicts
    
    # Data structure for tabu list
    tabu_list = {}  # {(vertex, color): expiration_iteration}
    
    # Pre-compute adjacency lists
    neighbors = {v: graph.get_neighbors(v) for v in graph.adjacency_list}
    
    conflict_cache = {}
    iteration = 0
    consecutive_no_improvement = 0
    diversification_threshold = 50  # Threshold for diversification
    
    while iteration < max_iterations and best_conflicts > 0:
        best_move = None
        best_move_conflicts = float('inf')
        
        # Identify conflicting vertices for a more targeted search
        conflicting = get_conflicting_vertices(graph, current_coloring, conflict_cache)
        
        # If no conflicting vertices but conflicts exist
        if not conflicting and current_conflicts > 0:
            conflicting = set(graph.adjacency_list.keys())
        
        # Examine moves for conflicting vertices
        for v in conflicting:
            current_color = current_coloring[v]
            
            for new_color in range(1, initial_k + 1):
                if new_color != current_color:
                    # Efficient calculation of conflicts after the move
                    delta_e = 0
                    for neighbor in neighbors[v]:
                        if current_coloring[neighbor] == current_color:
                            delta_e -= 1  # Removing a conflict
                        if current_coloring[neighbor] == new_color:
                            delta_e += 1  # Adding a conflict
                    
                    new_conflicts = current_conflicts + delta_e
                    
                    # Check if the move is tabu
                    is_tabu = (v, new_color) in tabu_list and tabu_list[(v, new_color)] > iteration
                    
                    # Aspiration criterion or non-tabu move
                    if (not is_tabu) or new_conflicts < best_conflicts:
                        if new_conflicts < best_move_conflicts:
                            best_move = (v, new_color)
                            best_move_conflicts = new_conflicts
        
        # Apply the best move or diversify if stagnation
        if best_move:
            v, new_color = best_move
            old_color = current_coloring[v]
            current_coloring[v] = new_color
            current_conflicts = best_move_conflicts
            
            # Update conflict cache
            conflict_cache.pop(id(current_coloring), None)
            
            # Add the reverse move to the tabu list with variable duration
            tabu_tenure_adjust = min(10, max(5, best_conflicts // 10))  # Dynamic adjustment
            tabu_list[(v, old_color)] = iteration + tabu_tenure + tabu_tenure_adjust
            
            # Update the best solution if necessary
            if current_conflicts < best_conflicts:
                best_coloring = current_coloring.copy()
                best_conflicts = current_conflicts
                consecutive_no_improvement = 0
            else:
                consecutive_no_improvement += 1
        else:
            consecutive_no_improvement += 1
        
        # Diversification strategy
        if consecutive_no_improvement >= diversification_threshold:
            # Diversify the current solution
            for i, v in enumerate(current_coloring):
                if i % 3 == 0:  # Modify one third of vertices
                    current_coloring[v] = random.randint(1, initial_k)
            
            current_conflicts = count_conflicts(graph, current_coloring)
            conflict_cache.pop(id(current_coloring), None)
            consecutive_no_improvement = 0
        
        iteration += 1
    
    return best_coloring, best_conflicts

In [14]:
def vns(graph, initial_coloring=None, initial_k=None, max_iterations=1000, k_max=3):
    """
    Optimized variable neighborhood search (VNS) method for graph coloring.
    """
    # Initialization
    if initial_coloring is None:
        if initial_k is None:
            initial_coloring = dsatur_coloring(graph)
            initial_k = max(initial_coloring.values())
        else:
            initial_coloring = {v: random.randint(1, initial_k) for v in graph.adjacency_list}
    
    current_coloring = initial_coloring.copy()
    current_conflicts = count_conflicts(graph, current_coloring)
    
    best_coloring = current_coloring.copy()
    best_conflicts = current_conflicts
    
    # Pre-compute adjacency lists
    neighbors = {v: graph.get_neighbors(v) for v in graph.adjacency_list}
    
    conflict_cache = {}
    iteration = 0
    improvement_threshold = 50  # Threshold for trying color reduction
    last_improvement = 0
    
    while iteration < max_iterations and best_conflicts > 0:
        k = 1  # Index of the neighborhood structure
        
        while k <= k_max:
            # Generate a solution in neighborhood k
            new_coloring = shake(graph, current_coloring, k, initial_k, neighbors, conflict_cache)
            
            # More efficient local search
            local_coloring, local_conflicts = local_search(graph, new_coloring, initial_k, neighbors, conflict_cache)
            
            # Check if the local solution is better
            if local_conflicts < current_conflicts:
                current_coloring = local_coloring.copy()
                current_conflicts = local_conflicts
                k = 1  # Reset neighborhood index
                last_improvement = iteration
            else:
                k += 1  # Move to next neighborhood
            
            # Update best solution if necessary
            if current_conflicts < best_conflicts:
                best_coloring = current_coloring.copy()
                best_conflicts = current_conflicts
        
        # Try color reduction periodically
        if iteration - last_improvement > improvement_threshold and best_conflicts == 0:
            try_reduce_colors = True
            current_coloring = best_coloring.copy()
            current_conflicts = best_conflicts
            last_improvement = iteration
        
        iteration += 1
    
    return best_coloring, best_conflicts

### Solver Function

The `solve_graph_coloring` function provides a unified interface to apply different coloring methods, including heuristics and metaheuristics, with a time limit to optimize the number of colors used.

In [15]:

def solve_graph_coloring(graph, method='dsatur_sa', initial_k=None, time_limit=60):
    """
    Solves the graph coloring problem using the specified method.
    """
    start_time = time.time()
    best_coloring = {}
    best_k = float('inf')
    conflict_cache = {}

    # Run initial algorithm
    if method in ['greedy', 'welsh_powell', 'dsatur']:
        if method == 'greedy':
            coloring = greedy_coloring(graph)
        elif method == 'welsh_powell':
            coloring = welsh_powell_coloring(graph)
        else:  # dsatur
            coloring = dsatur_coloring(graph)
        
        return max(coloring.values()), coloring

    elif method in ['dsatur_sa', 'dsatur_tabu', 'dsatur_vns']:
        coloring = dsatur_coloring(graph)
        initial_k = max(coloring.values())

        vertex_count = len(graph.adjacency_list)
        max_no_improve = 20
        no_improve_counter = 0
        last_k = initial_k

        best_coloring = coloring
        best_k = initial_k

        if method == 'dsatur_sa':
            iterations = min(10000, vertex_count * 100)
            initial_temp = 1.0 if vertex_count < 100 else 10.0
            cooling_rate = 0.99 if vertex_count < 100 else 0.995

            while time.time() - start_time < time_limit:
                current_coloring, conflicts = simulated_annealing(
                    graph, coloring, initial_k, iterations, initial_temp, cooling_rate
                )

                if conflicts == 0:
                    current_k = max(current_coloring.values())
                    if current_k < best_k:
                        best_k = current_k
                        best_coloring = current_coloring.copy()
                        no_improve_counter = 0
                    else:
                        no_improve_counter += 1
                else:
                    no_improve_counter += 1

                if best_k > 1:
                    initial_k = best_k - 1
                    coloring = {v: random.randint(1, initial_k) for v in graph.adjacency_list}
                else:
                    break

                if no_improve_counter >= max_no_improve:
                    break

        elif method == 'dsatur_tabu':
            iterations = min(1000, vertex_count * 50)
            tabu_tenure = max(5, vertex_count // 20)

            while time.time() - start_time < time_limit:
                current_coloring, conflicts = tabu_search(
                    graph, coloring, initial_k, iterations, tabu_tenure
                )

                if conflicts == 0:
                    current_k = max(current_coloring.values())
                    if current_k < best_k:
                        best_k = current_k
                        best_coloring = current_coloring.copy()
                        no_improve_counter = 0
                    else:
                        no_improve_counter += 1
                else:
                    no_improve_counter += 1

                if best_k > 1:
                    initial_k = best_k - 1
                    coloring = {v: random.randint(1, initial_k) for v in graph.adjacency_list}
                else:
                    break

                if no_improve_counter >= max_no_improve:
                    break

        elif method == 'dsatur_vns':
            iterations = min(1000, vertex_count * 30)
            k_max = 3 if vertex_count < 100 else 4

            while time.time() - start_time < time_limit:
                current_coloring, conflicts = vns(
                    graph, coloring, initial_k, iterations, k_max
                )

                if conflicts == 0:
                    current_k = max(current_coloring.values())
                    if current_k < best_k:
                        best_k = current_k
                        best_coloring = current_coloring.copy()
                        no_improve_counter = 0
                    else:
                        no_improve_counter += 1
                else:
                    no_improve_counter += 1

                if best_k > 1:
                    initial_k = best_k - 1
                    coloring = {v: random.randint(1, initial_k) for v in graph.adjacency_list}
                else:
                    break

                if no_improve_counter >= max_no_improve:
                    break

        return best_k, best_coloring

    else:
        raise ValueError(f"Method not recognized: {method}")

# Graph Coloring Algorithms Implementation : Approach 2
### Initialization
- **DSATUR Algorithm**: 
  - Orders vertices by saturation degree (number of distinct colors in neighborhood)
  - Ties broken by vertex degree
  - Provides high-quality starting solutions

### Evaluation Metrics
- **Conflict Detection**: Efficiently counts edge violations
- **Objective Function**: 
  ```python
  objective = num_colors + α * conflicts
  ```
  - Strongly penalizes conflicts while minimizing color count

## Optimization Algorithms

### 1. Simulated Annealing (SA)
- **Key Features**:
  - Temperature-based probabilistic acceptance of worse solutions
  - Adaptive neighborhood selection:
    - Prefers conflict-free moves when possible
    - Allows color reduction attempts
  - Cooling schedule: Geometric decay from 100° to 0.01°

### 2. Tabu Search
- **Advanced Mechanisms**:
  - Dynamic tabu tenure with random variation
  - Frequency-based diversification
  - Aspiration criteria to override tabu status
  - Focused search on conflicting vertices

### Initialization Function

The `dsatur_initialization` function implements the DSatur (Degree of Saturation) algorithm to generate an initial coloring. It selects vertices based on their saturation degree (number of unique neighbor colors) and assigns the smallest possible color not used by neighbors.

In [16]:
def dsatur_initialization(graph):
    """
    Initializes a coloring using the DSatur algorithm.
    
    Args:
        graph: Graph object with adjacency_list and get_neighbors method
        
    Returns:
        dict: A coloring mapping vertices to colors
    """
    coloring = {}  # Coloring: {vertex: color}
    uncolored = set(graph.adjacency_list.keys())
    saturation = {v: 0 for v in uncolored}  # Saturation degree of each vertex
    
    # Function to update the saturation degree of a vertex
    def update_saturation(vertex):
        adjacent_colors = set()
        for neighbor in graph.get_neighbors(vertex):
            if neighbor in coloring:
                adjacent_colors.add(coloring[neighbor])
        saturation[vertex] = len(adjacent_colors)
    
    # Initialize saturation degrees
    for v in uncolored:
        update_saturation(v)
    
    while uncolored:
        # Select the uncolored vertex with the highest saturation degree
        # In case of a tie, choose the one with the highest degree
        selected = max(uncolored, key=lambda v: (saturation[v], graph.get_degree(v)))
        
        # Find the smallest available color
        used_colors = set()
        for neighbor in graph.get_neighbors(selected):
            if neighbor in coloring:
                used_colors.add(coloring[neighbor])
        
        color = 1
        while color in used_colors:
            color += 1
        
        # Assign the color
        coloring[selected] = color
        uncolored.remove(selected)
        
        # Update saturation degrees of uncolored neighbors
        for neighbor in graph.get_neighbors(selected):
            if neighbor in uncolored:
                update_saturation(neighbor)
    
    return coloring

### Evaluation Functions

These functions evaluate and guide the coloring process:
- `evaluate_solution`: Counts the number of colors and conflicts in a coloring.
- `calculate_objective`: Combines the number of colors and conflicts into a single objective value, with a high penalty for conflicts.
- `compute_conflict_vertices`: Identifies vertices involved in conflicts, used to focus the search in Tabu Search.

In [17]:
def evaluate_solution(graph, coloring):
    """
    Evaluates a coloring solution by counting the number of conflicts
    and the number of colors used.
    
    Args:
        graph: Graph object with adjacency_list and get_neighbors method
        coloring: Dictionary mapping vertices to colors
        
    Returns:
        tuple: (number of colors, number of conflicts)
    """
    # Calculate the number of colors
    num_colors = max(coloring.values()) if coloring else 0
    
    # Count conflicts
    conflicts = 0
    counted_edges = set()
    
    for u in graph.adjacency_list:
        for v in graph.get_neighbors(u):
            if u < v:  # For each edge, count it only once
                edge = (u, v) if u < v else (v, u)
                if edge not in counted_edges:
                    counted_edges.add(edge)
                    if coloring.get(u) == coloring.get(v):
                        conflicts += 1
    
    return num_colors, conflicts

def calculate_objective(num_colors, conflicts, alpha=1000):
    """
    Calculates the objective function that combines the number of colors and conflicts.
    
    Args:
        num_colors: Number of colors used
        conflicts: Number of conflicts
        alpha: Weighting factor for conflicts (typically high)
    
    Returns:
        float: Objective value (lower is better)
    """
    return num_colors + alpha * conflicts

def compute_conflict_vertices(graph, coloring):
    """
    Computes the set of vertices involved in conflicts.
    
    Args:
        graph: Graph object with adjacency_list and get_neighbors method
        coloring: Dictionary mapping vertices to colors
        
    Returns:
        set: Set of vertices involved in conflicts
    """
    conflict_vertices = set()
    for u in graph.adjacency_list:
        for v in graph.get_neighbors(u):
            if u < v and coloring[u] == coloring[v]:
                conflict_vertices.add(u)
                conflict_vertices.add(v)
    return conflict_vertices

In [18]:
def simulated_annealing_integrated(graph, max_iterations=10000, initial_temp=100.0, 
                                  cooling_rate=0.995, min_temp=0.01):
    """
    Simulated annealing method that directly integrates the minimization of the number
    of colors into the objective function.
    
    Args:
        graph: Graph object with adjacency_list and get_neighbors method
        max_iterations: Maximum number of iterations
        initial_temp: Initial temperature
        cooling_rate: Cooling rate
        min_temp: Minimum temperature
        
    Returns:
        tuple: (best coloring, best number of colors, best number of conflicts, iteration stats)
    """
    # Initialize with DSatur
    current_coloring = dsatur_initialization(graph)
    num_colors, conflicts = evaluate_solution(graph, current_coloring)
    current_objective = calculate_objective(num_colors, conflicts)
    
    best_coloring = current_coloring.copy()
    best_objective = current_objective
    best_num_colors = num_colors
    best_conflicts = conflicts
    
    temperature = initial_temp
    
    # Prepare statistics tracking
    iteration_stats = []
    
    for iteration in range(max_iterations):
        if temperature < min_temp:
            break
            
        # Select a random vertex
        vertex = random.choice(list(graph.adjacency_list.keys()))
        old_color = current_coloring[vertex]
        
        # Generate a move
        # Determine the range of possible colors
        max_color = max(current_coloring.values())
        
        if conflicts == 0 and random.random() < 0.7:
            # If no conflicts, try to reduce the number of colors
            available_colors = set(range(1, max_color))  # Existing colors except the highest
        else:
            # Otherwise, allow introducing a new color with a small probability
            available_colors = set(range(1, max_color + 2))  # +1 for new color
        
        available_colors.discard(old_color)  # Exclude current color
        
        # If there are available colors not used by neighbors, prioritize them
        neighbor_colors = {current_coloring[n] for n in graph.get_neighbors(vertex)}
        preferred_colors = available_colors - neighbor_colors
        
        if preferred_colors and random.random() < 0.8:
            new_color = random.choice(list(preferred_colors))
        elif available_colors:
            new_color = random.choice(list(available_colors))
        else:
            continue  # No possible move
        
        # Apply move temporarily
        current_coloring[vertex] = new_color
        
        # Evaluate new solution
        new_num_colors, new_conflicts = evaluate_solution(graph, current_coloring)
        new_objective = calculate_objective(new_num_colors, new_conflicts)
        
        # Decide whether to accept the move
        delta = new_objective - current_objective
        
        if delta <= 0 or random.random() < math.exp(-delta / temperature):
            # Accept the move
            current_objective = new_objective
            num_colors = new_num_colors
            conflicts = new_conflicts
            
            # Update best solution if necessary
            if (new_conflicts < best_conflicts) or (new_conflicts == best_conflicts and new_num_colors < best_num_colors):
                best_coloring = current_coloring.copy()
                best_objective = new_objective
                best_num_colors = new_num_colors
                best_conflicts = new_conflicts
        else:
            # Reject the move
            current_coloring[vertex] = old_color
        
        # Cool the temperature
        temperature *= cooling_rate
        
        # Record statistics for this iteration
        if iteration % 100 == 0:
            iteration_stats.append((iteration, num_colors, conflicts, temperature))
    
    return best_coloring, best_num_colors, best_conflicts, iteration_stats

In [19]:
def tabu_search_integrated(graph, max_iterations=5000, tabu_tenure=10, 
                          aspiration_factor=0.9, diversification_threshold=100):
    """
    Tabu search method that directly integrates the minimization of the number
    of colors into the objective function.
    
    Args:
        graph: Graph object with adjacency_list and get_neighbors method
        max_iterations: Maximum number of iterations
        tabu_tenure: Base tabu tenure
        aspiration_factor: Factor for aspiration criterion
        diversification_threshold: Threshold for diversification
        
    Returns:
        tuple: (best coloring, best number of colors, best number of conflicts, iteration stats)
    """
    # Initialize with DSatur
    current_coloring = dsatur_initialization(graph)
    num_colors, conflicts = evaluate_solution(graph, current_coloring)
    current_objective = calculate_objective(num_colors, conflicts)
    
    best_coloring = current_coloring.copy()
    best_objective = current_objective
    best_num_colors = num_colors
    best_conflicts = conflicts
    
    # Initialize data structures for tabu search
    tabu_list = {}  # {(vertex, color): expiration_iteration}
    frequency = defaultdict(int)  # {(vertex, color): frequency}
    
    # Prepare statistics tracking
    iteration_stats = []
    stagnation_counter = 0
    last_improvement = 0
    
    vertices = list(graph.adjacency_list.keys())
    
    for iteration in range(max_iterations):
        # Determine vertices to consider
        if conflicts > 0:
            # If there are conflicts, focus on conflicting vertices
            conflict_vertices = compute_conflict_vertices(graph, current_coloring)
            candidates = list(conflict_vertices) if conflict_vertices else vertices
        else:
            # If there are no conflicts, try to reduce the number of colors
            max_color = max(current_coloring.values())
            candidates = [v for v in vertices if current_coloring[v] == max_color]
            if not candidates:  # If no vertex has the maximum color
                candidates = vertices
        
        # Randomly select a subset of candidate vertices for efficiency
        if len(candidates) > 50:
            candidates = random.sample(candidates, 50)
        
        best_move = None
        best_move_objective = float('inf')
        
        # Evaluate all possible moves for candidate vertices
        for vertex in candidates:
            old_color = current_coloring[vertex]
            max_color = max(current_coloring.values())
            
            # Determine colors to try
            if conflicts == 0:
                # If no conflicts, try to reduce the number of colors
                colors_to_try = set(range(1, max_color))  # All colors except the highest
            else:
                # Otherwise, allow adding a new color with a small probability
                colors_to_try = set(range(1, max_color + (1 if random.random() < 0.05 else 0) + 1))
            
            # Exclude current color
            colors_to_try.discard(old_color)
            
            # Evaluate each possible color
            for new_color in colors_to_try:
                # Check if this move is tabu
                is_tabu = tabu_list.get((vertex, new_color), 0) > iteration
                
                # Simulate the move
                current_coloring[vertex] = new_color
                new_num_colors, new_conflicts = evaluate_solution(graph, current_coloring)
                new_objective = calculate_objective(new_num_colors, new_conflicts)
                
                # Apply aspiration criterion if necessary
                aspiration_condition = new_objective < best_objective * aspiration_factor
                
                if (not is_tabu or aspiration_condition) and new_objective < best_move_objective:
                    best_move = (vertex, new_color)
                    best_move_objective = new_objective
                
                # Undo simulation
                current_coloring[vertex] = old_color
        
        # If no move is found, diversify
        if best_move is None:
            # Apply perturbation to escape local optimum
            for _ in range(min(10, len(vertices))):
                v = random.choice(vertices)
                old_color = current_coloring[v]
                max_color = max(current_coloring.values())
                
                # Choose a different color
                new_color = random.randint(1, max_color + (1 if random.random() < 0.1 else 0))
                while new_color == old_color:
                    new_color = random.randint(1, max_color + (1 if random.random() < 0.1 else 0))
                
                current_coloring[v] = new_color
            
            # Re-evaluate after diversification
            num_colors, conflicts = evaluate_solution(graph, current_coloring)
            current_objective = calculate_objective(num_colors, conflicts)
            stagnation_counter = 0
            continue
        
        # Apply best move
        vertex, new_color = best_move
        old_color = current_coloring[vertex]
        current_coloring[vertex] = new_color
        
        # Update current values
        num_colors, conflicts = evaluate_solution(graph, current_coloring)
        current_objective = calculate_objective(num_colors, conflicts)
        
        # Update tabu list
        tabu_list[(vertex, old_color)] = iteration + tabu_tenure + random.randint(-2, 2)  # Random variation
        
        # Update frequency
        frequency[(vertex, new_color)] += 1
        
        # Update best solution if necessary
        if (conflicts < best_conflicts) or (conflicts == best_conflicts and num_colors < best_num_colors):
            best_coloring = current_coloring.copy()
            best_objective = current_objective
            best_num_colors = num_colors
            best_conflicts = conflicts
            last_improvement = iteration
            stagnation_counter = 0
        else:
            stagnation_counter += 1
        
        # Diversification if stagnation
        if stagnation_counter >= diversification_threshold:
            # Diversification strategy based on frequency
            if stagnation_counter >= 2 * diversification_threshold:
                # Strong diversification: partial reset based on frequency
                for v in random.sample(vertices, max(5, len(vertices) // 10)):
                    # Choose a color with low usage frequency
                    colors_freq = [(color, frequency.get((v, color), 0)) 
                                  for color in range(1, max(current_coloring.values()) + 1)]
                    colors_freq.sort(key=lambda x: x[1])  # Sort by frequency
                    if colors_freq:
                        low_freq_colors = [c for c, f in colors_freq[:min(3, len(colors_freq))]]
                        if low_freq_colors:
                            current_coloring[v] = random.choice(low_freq_colors)
            else:
                # Moderate diversification: random modification of some vertices
                for _ in range(min(5, len(vertices))):
                    v = random.choice(vertices)
                    max_color = max(current_coloring.values())
                    current_coloring[v] = random.randint(1, max_color)
            
            # Re-evaluate after diversification
            num_colors, conflicts = evaluate_solution(graph, current_coloring)
            current_objective = calculate_objective(num_colors, conflicts)
            stagnation_counter = 0
        
        # Record statistics for this iteration
        if iteration % 100 == 0:
            iteration_stats.append((iteration, num_colors, conflicts))
    
    return best_coloring, best_num_colors, best_conflicts, iteration_stats

# Benchmarks
### Benchmark Graphs
- Tests against standard DIMACS graph coloring instances
- Includes known optimal/reference values for:
  - `dsjc250.5` (K=28)
  - `dsjc500.9` (K=126)  
  - `dsjr500.1c` (K=84, optimal)
  - `flat1000_76_0` (K=76, optimal)
  - `r250.5` (K=65, optimal)

### Tested Algorithms
1. **DSATUR-based Hybrids**:
   - `dsatur_sa`: DSATUR + Simulated Annealing
   - `dsatur_tabu`: DSATUR + Tabu Search
   - `dsatur_vns`: DSATUR + Variable Neighborhood Search

2. **Integrated Methods**:
   - `simulated_annealing_integrated`: SA with color minimization
   - `tabu_search_integrated`: Tabu with adaptive strategies

### Evaluation Metrics
- **Solution Quality**:
  - Number of colors (K)
  - Gap from known best (%)
  - Number of conflicts
  - Validity check (conflicts == 0)

- **Performance**:
  - Execution time (seconds)
  - Color distribution analysis

In [20]:
def test_coloring_procedures(graph_files, time_limit=300):
    """
    Test different graph coloring procedures on a list of graph files.
    
    Args:
        graph_files: Dictionary {graph_name: file_path}
        time_limit: Time limit in seconds for each execution
        
    Returns:
        DataFrame pandas with the results
    """
    import pandas as pd
    import time
    from collections import defaultdict
    import random
    
    # Define known best K for certain graphs
    known_best_k = {
        'dsjc250.5': 28,  # Reference, not necessarily optimal
        'dsjc500.9': 126,  # Reference, not necessarily optimal
        'dsjr500.1c': 84,  # Known optimal
        'flat1000_76_0': 76,  # Known optimal
        'r250.5': 65  # Known optimal
    }
    
    # Define methods to test
    methods_to_test = [
        'dsatur_sa',
        'dsatur_tabu',
        'dsatur_vns',
        'simulated_annealing_integrated',
        'tabu_search_integrated',
    ]
    
    # Prepare DataFrame for results
    results = []
    
    # For each graph
    for graph_name, graph_file in graph_files.items():
        print(f"\nProcessing graph: {graph_name}")
        
        # Load the graph
        try:
            graph = Graph.from_dimacs(graph_file)
            print(f"Graph loaded: {graph.num_vertices} vertices, {graph.num_edges} edges")
        except Exception as e:
            print(f"Error loading graph {graph_name}: {e}")
            continue
        
        # For each method
        for method in methods_to_test:
            print(f"Testing method: {method}")
            
            # Execute the algorithm
            start_time = time.time()
            try:
                if method in ['dsatur_sa', 'dsatur_tabu', 'dsatur_vns']:
                    k, coloring = solve_graph_coloring(graph, method=method, time_limit=time_limit)
                    conflicts = count_conflicts(graph, coloring)
                elif method == 'simulated_annealing_integrated':
                    coloring, k, conflicts, _ = simulated_annealing_integrated(
                        graph,
                        max_iterations=50000,
                        initial_temp=100.0,
                        cooling_rate=0.995
                    )
                elif method == 'tabu_search_integrated':
                    coloring, k, conflicts, _ = tabu_search_integrated(
                        graph,
                        max_iterations=3500,
                        tabu_tenure=10,
                        diversification_threshold=100
                    )
                else:
                    raise ValueError(f"Method not recognized: {method}")
                    
            except Exception as e:
                print(f"Error executing {method} on {graph_name}: {e}")
                k = float('inf')
                conflicts = float('inf')
                coloring = {}
                
            execution_time = time.time() - start_time
            
            # Calculate gap from known best K
            known_best = known_best_k.get(graph_name, float('inf'))
            gap = ((k - known_best) / known_best * 100) if known_best < float('inf') and k < float('inf') else float('inf')
            
            # Analyze color usage
            color_usage = defaultdict(int)
            for color in coloring.values():
                color_usage[color] += 1
            
            color_distribution = {color: count for color, count in sorted(color_usage.items())}
            
            # Add results
            result = {
                'Graph': graph_name,
                'Method': method,
                'K': k,
                'Known Best K': known_best,
                'Gap (%)': gap if gap != float('inf') else 'N/A',
                'Conflicts': conflicts,
                'Time (s)': execution_time,
                'Valid Coloring': conflicts == 0,
                'Color Distribution': str(color_distribution)
            }
            results.append(result)
            
            print(f"Result: K={k}, Conflicts={conflicts}, Time={execution_time:.2f}s, Valid={conflicts==0}")
    
    # Create DataFrame
    results_df = pd.DataFrame(results)
    
    # Display results
    print("\nComplete results:")
    print(results_df[['Graph', 'Method', 'K', 'Known Best K', 'Conflicts', 'Time (s)', 'Valid Coloring']])
    
    return results_df


In [ ]:
def main():
    """
    Main function to test different graph coloring procedures on benchmark instances.
    """
    import matplotlib.pyplot as plt
    
    # Define paths to graph files
    graph_files = {
        'r250.5': '/kaggle/input/optimisation/r250.5.col',
        'dsjc250.5': '/kaggle/input/optimisation/dsjc250.5.col',
    }
    # Run the tests
    results = test_coloring_procedures(graph_files)
    
    # Save results to CSV
    results.to_csv('coloring_results.csv', index=False)
    
    # Generate visualizations
    plt.figure(figsize=(12, 8))
    
    # Graph of K values by method and graph
    plt.subplot(2, 1, 1)
    valid_results = results[results['Valid Coloring'] == True]
    
    for graph in valid_results['Graph'].unique():
        graph_data = valid_results[valid_results['Graph'] == graph]
        methods = graph_data['Method'].values
        ks = graph_data['K'].values
        plt.bar(methods, ks, label=graph, alpha=0.7)
    
    plt.xlabel('Method')
    plt.ylabel('Number of colors (K)')
    plt.title('Comparison of K values by method and graph')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Graph of execution times
    plt.subplot(2, 1, 2)
    for graph in valid_results['Graph'].unique():
        graph_data = valid_results[valid_results['Graph'] == graph]
        methods = graph_data['Method'].values
        times = graph_data['Time (s)'].values
        plt.bar(methods, times, label=graph, alpha=0.7)
    
    plt.xlabel('Method')
    plt.ylabel('Execution time (s)')
    plt.title('Comparison of execution times by method and graph')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig('coloring_comparison.png')
    
    print("Results saved to coloring_results.csv")
    print("Visualizations saved to coloring_comparison.png")


main()


Processing graph: r250.5
Graph loaded: 250 vertices, 14849 edges
Testing method: dsatur_sa
Result: K=68, Conflicts=0, Time=14.00s, Valid=True
Testing method: dsatur_tabu
